In [1]:
import sys

sys.path.append("../scripts")
from pathlib import Path

from tqdm.auto import tqdm

import pandas as pd
from sklearn.decomposition import PCA, KernelPCA

In [2]:
profiles = {
    "subset->preprocess": "../1.load/output/subset_processed_profiles.parquet",
    "subset->cc_adjust->preprocess": "../3.correct/output/subset_profiles_cc_corrected.parquet",
    "subset->well_correct->preprocess": "../3.correct/output/subset_profiles_mean_corrected_preprocessed.parquet",
    "subset->cc_adjust->well_correct->preprocess": "../3.correct/output/subset_profiles_cc_well_corrected.parquet",
}

In [3]:
df = pd.read_parquet(profiles["subset->preprocess"])
df

,Metadata_JCP2022,Metadata_broad_sample,Metadata_Name,Metadata_Vector,Metadata_Transcript,Metadata_Symbol,Metadata_NCBI_Gene_ID,Metadata_Taxon_ID,Metadata_Gene_Description,Metadata_Prot_Match,...,Nuclei_Texture_InfoMeas1_ER_3_03_256,Nuclei_Texture_InfoMeas1_Mito_3_02_256,Nuclei_Texture_InfoMeas1_RNA_3_00_256,Nuclei_Texture_InfoMeas2_AGP_3_00_256,Nuclei_Texture_InfoMeas2_ER_3_00_256,Nuclei_Texture_InfoMeas2_Mito_3_00_256,Nuclei_Texture_SumVariance_AGP_5_01_256,Nuclei_Texture_SumVariance_DNA_3_02_256,Nuclei_Texture_SumVariance_ER_10_03_256,Cells_Count
0,JCP2022_900008,ccsbBroad304_00010,ORF011294.1_TRC304.1,pLX_304,NM_001278352.1,ACO1,48,9606,aconitase 1,100.0,...,-1.630447,0.320777,-1.428334,-0.046419,1.271371,-0.208324,0.202360,-0.044861,1.367060,1.016925
1,JCP2022_900034,ccsbBroad304_00038,ORF001874.1_TRC304.1,pLX_304,NM_000643.2,AGL,178,9606,"amylo-alpha-1, 6-glucosidase, 4-alpha-glucanot...",100.0,...,-1.713351,-0.090217,-1.458614,0.120309,1.279027,0.650121,0.918840,-1.644120,0.823474,1.577271
2,JCP2022_900327,ccsbBroad304_00356,ORF005713.1_TRC304.1,pLX_304,NM_001304.5,CPD,1362,9606,carboxypeptidase D,100.0,...,-0.825970,-0.270656,-1.012781,1.297825,0.836286,0.252346,0.643505,0.677639,0.375317,-0.352811
3,JCP2022_900417,ccsbBroad304_00448,ORF010275.1_TRC304.1,pLX_304,NM_021120.4,DLG3,1741,9606,discs large MAGUK scaffold protein 3,100.0,...,0.986662,-0.681650,-0.306966,1.863373,-0.466784,2.045490,-1.688061,-2.013634,-1.290032,-0.290550
4,JCP2022_900443,ccsbBroad304_00474,ORF014835.1_TRC304.1,pLX_304,NM_001258316.1,ECT2,1894,9606,epithelial cell transforming 2,100.0,...,-0.172973,-0.744660,0.122160,1.553597,-0.090734,0.735807,0.096500,0.539514,-0.416065,-0.477332
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19138,JCP2022_915130,ccsbBroad304_99991,ORFC00003.1_TRC304.1,pLX_304,promegaLuc.1,LUCIFERASE,LUCIFERASE,CONTROL,promegaLuc,100.0,...,-0.553293,0.229975,-0.506501,0.138817,0.536639,-0.447200,0.750188,-0.106818,0.762016,0.208088
19139,JCP2022_915131,ccsbBroad304_99994,ORFC00004.1_TRC304.1,pLX_304,LacZ.1,LacZ,LacZ,CONTROL,Hahn Lab LacZ,100.0,...,0.866828,1.257922,0.440656,-0.330972,-0.730616,-0.157921,-0.383947,0.371311,-0.588184,1.270052
19140,JCP2022_915131,ccsbBroad304_99994,ORFC00004.1_TRC304.1,pLX_304,LacZ.1,LacZ,LacZ,CONTROL,Hahn Lab LacZ,100.0,...,1.806110,0.192932,1.315214,-1.499569,-1.646347,-0.357171,-1.256609,-0.274468,-1.299471,1.076315
19141,JCP2022_915131,ccsbBroad304_99994,ORFC00004.1_TRC304.1,pLX_304,LacZ.1,LacZ,LacZ,CONTROL,Hahn Lab LacZ,100.0,...,-0.538802,-0.879772,-0.753843,0.676057,0.616393,-0.898091,-0.657668,0.375204,-0.104637,-1.191122


In [4]:
# choose Metadata_Sybmbol values that have more than 1 unique Metadata_Batch values
group_feature = "Metadata_JCP2022"
groups = df[df.Metadata_pert_type == "trt"].groupby(group_feature).filter(lambda x: len(x.Metadata_Well.unique()) > 1).groupby(group_feature)


# iterate over groups and print "Metadata_Batch" values
for name, group in groups:
    print(name)
    print(group[["Metadata_Batch", "Metadata_Plate", "Metadata_Well"]])
    print()

JCP2022_900041
          Metadata_Batch Metadata_Plate Metadata_Well
6670   2021_06_14_Batch6     BR00123945           F08
8794   2021_06_14_Batch6     BR00124766           F08
9148   2021_06_14_Batch6     BR00124767           F08
9502   2021_06_14_Batch6     BR00124768           F08
9856   2021_06_14_Batch6     BR00124769           F08
10921  2021_07_12_Batch8     BR00124787           H13
11281  2021_07_12_Batch8     BR00124788           H13
12703  2021_07_12_Batch8     BR00125619           H13
13062  2021_07_12_Batch8     BR00125620           H13
13422  2021_07_12_Batch8     BR00125621           H13

JCP2022_900288
         Metadata_Batch Metadata_Plate Metadata_Well
1769  2021_05_31_Batch2     BR00121552           G21
2037  2021_05_31_Batch2     BR00121553           G21
2305  2021_05_31_Batch2     BR00121554           G21
2573  2021_05_31_Batch2     BR00121555           G21
2841  2021_05_31_Batch2     BR00121556           G21
6671  2021_06_14_Batch6     BR00123945           P06
8795

In [5]:
group_feature = "Metadata_Symbol"
groups = df[df.Metadata_pert_type == "trt"].groupby(group_feature).filter(lambda x: len(x.Metadata_Well.unique()) > 1).groupby(group_feature)

unique_symbols = groups["Metadata_Batch"].nunique()
unique_symbols.value_counts()

2    172
1     89
3      3
Name: Metadata_Batch, dtype: int64

In [6]:
for name, group in groups:
    print(name)
    print(group[["Metadata_JCP2022", "Metadata_Batch", "Metadata_Plate", "Metadata_Well"]])
    print()

AAK1
      Metadata_JCP2022     Metadata_Batch Metadata_Plate Metadata_Well
6915    JCP2022_914015  2021_06_14_Batch6     BR00123945           B21
9039    JCP2022_914015  2021_06_14_Batch6     BR00124766           B21
9393    JCP2022_914015  2021_06_14_Batch6     BR00124767           B21
9747    JCP2022_914015  2021_06_14_Batch6     BR00124768           B21
10101   JCP2022_914015  2021_06_14_Batch6     BR00124769           B21
11250   JCP2022_914725  2021_07_12_Batch8     BR00124787           J07
11610   JCP2022_914725  2021_07_12_Batch8     BR00124788           J07
13031   JCP2022_914725  2021_07_12_Batch8     BR00125619           J07
13391   JCP2022_914725  2021_07_12_Batch8     BR00125620           J07
13750   JCP2022_914725  2021_07_12_Batch8     BR00125621           J07

AASDHPPT
      Metadata_JCP2022     Metadata_Batch Metadata_Plate Metadata_Well
3191    JCP2022_903653  2021_05_10_Batch3     BR00123613           N19
3549    JCP2022_903653  2021_05_10_Batch3     BR00123614      

In [7]:
metrics = pd.read_parquet("../2.evalzoo/results/26efacc8/metrics_level_1_ref.parquet")
metrics["p<0.05"] = metrics["sim_retrieval_average_precision_ref_i_nlog10pvalue_mean_i"] > 1.3
metrics

KeyError: '-log(pvalue)'

In [ ]:
metrics.groupby("Metadata_Symbol")["sim_retrieval_average_precision_ref_i_mean_i"].mean().mean()

0.026971369627183

In [ ]:
unique_symbols.name = "Metadata_Batch_Count"
unique_symbols

Metadata_Symbol
AAK1              2
AASDHPPT          2
ABHD17A           2
ADGRG7            1
ADK               2
                 ..
XLOC_l2_013458    2
ZCCHC9            1
ZFC3H1            2
ZGPAT             2
ZNF544            1
Name: Metadata_Batch_Count, Length: 264, dtype: int64

In [ ]:
metrics = metrics.merge(unique_symbols, left_on="Metadata_Symbol", right_index=True)
metrics

,Metadata_Symbol,Metadata_reference_or_other,Metadata_Well,sim_scaled_mean_ref_i_mean_i,sim_scaled_mean_ref_i_median_i,sim_scaled_median_ref_i_mean_i,sim_scaled_median_ref_i_median_i,sim_ranked_relrank_mean_ref_i_mean_i,sim_ranked_relrank_mean_ref_i_median_i,sim_ranked_relrank_median_ref_i_mean_i,...,sim_stat_background_n_ref_i_mean_i,sim_stat_background_n_ref_i_median_i,sim_retrieval_average_precision_ref_i_mean_i,sim_retrieval_average_precision_ref_i_median_i,sim_retrieval_r_precision_ref_i_mean_i,sim_retrieval_r_precision_ref_i_median_i,sim_retrieval_average_precision_ref_i_nlog10pvalue_mean_i,sim_retrieval_average_precision_ref_i_nlog10pvalue_median_i,sim_retrieval_average_precision_ref_i_nlog10qvalue_mean_i,Metadata_Batch_Count
0,AAK1,pert,B21,-0.407174,-0.476069,-0.431958,-0.479115,0.633205,0.655909,0.665909,...,880.0,880.0,0.005670,0.004963,0.00,0.0,0.120468,0.049778,0.043577,2
1,AAK1,pert,J07,-0.407174,-0.476069,-0.431958,-0.479115,0.633205,0.655909,0.665909,...,880.0,880.0,0.005670,0.004963,0.00,0.0,0.120468,0.049778,0.043577,2
2,AASDHPPT,pert,N19,-0.478581,-0.541485,-0.482861,-0.559909,0.685182,0.698182,0.698295,...,880.0,880.0,0.004868,0.004677,0.00,0.0,0.051561,0.030328,0.022773,2
3,AASDHPPT,pert,M10,-0.478581,-0.541485,-0.482861,-0.559909,0.685182,0.698182,0.698295,...,880.0,880.0,0.004868,0.004677,0.00,0.0,0.051561,0.030328,0.022773,2
4,ABHD17A,pert,D05,1.300405,1.268884,1.338076,1.274546,0.163068,0.183864,0.128523,...,880.0,880.0,0.060888,0.049500,0.02,0.0,1.418998,1.540934,0.379934,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
553,ZFC3H1,pert,F18,-0.667016,-0.798857,-0.851156,-1.070462,0.719614,0.756591,0.782614,...,880.0,880.0,0.005103,0.004266,0.00,0.0,0.083694,0.011136,0.037835,2
554,ZGPAT,pert,I17,0.119522,-0.009469,-0.111500,-0.446354,0.537932,0.581591,0.565114,...,880.0,880.0,0.011162,0.006091,0.00,0.0,0.273811,0.140741,0.078627,2
555,ZGPAT,pert,P06,0.119522,-0.009469,-0.111500,-0.446354,0.537932,0.581591,0.565114,...,880.0,880.0,0.011162,0.006091,0.00,0.0,0.273811,0.140741,0.078627,2
556,ZNF544,pert,N20,-0.124649,-0.038563,-0.067599,-0.041072,0.527432,0.507045,0.484205,...,880.0,880.0,0.006931,0.006628,0.00,0.0,0.219592,0.190317,0.064023,1


In [ ]:
metrics.groupby(["Metadata_Batch_Count", "Metadata_Symbol"])["sim_retrieval_average_precision_ref_i_mean_i"].mean()

Metadata_Batch_Count  Metadata_Symbol
1                     ADGRG7             0.028208
                      AGK                0.008979
                      AK4                0.005835
                      AKR1B1             0.015908
                      ALDH3B1            0.043964
                                           ...   
2                     ZFC3H1             0.005103
                      ZGPAT              0.011162
3                     ATM                0.044964
                      NSUN5              0.021682
                      PNKP               0.010090
Name: sim_retrieval_average_precision_ref_i_mean_i, Length: 264, dtype: float64

In [ ]:
metrics.groupby(["Metadata_Batch_Count"])["sim_retrieval_average_precision_ref_i_mean_i"].mean()

Metadata_Batch_Count
1    0.030124
2    0.027800
3    0.027517
Name: sim_retrieval_average_precision_ref_i_mean_i, dtype: float64

In [ ]:
for i in range(1,4):
    print(i, metrics.groupby(["Metadata_Batch_Count", "Metadata_Symbol"])["sim_retrieval_average_precision_ref_i_mean_i"].mean()[(i,)].mean())

1 0.03002525135899193
2 0.025415452761042203
3 0.025578778575590475
